# Algoritmo de Optimización _Firefly_
El algoritmo Firefly (FA) para optimización fue creado por Yang en el año 2009 como una propuesta de un
algoritmo de optimización _sin restricciones_, _multimodal_, biológicamente inspirado en el comportamiento
de las luciérnagas donde un grupo de luciérnagas es atraído hacia un punto fijo dependiendo de la intensidad
del brillo que emanan.

A diferencia de los algoritmos clásicos heurísticos como [Nelder-Mead](https://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method), [_Particle Swarm Optimization_](https://en.wikipedia.org/wiki/Particle_swarm_optimization), 
[_Simulated Annealing_](https://en.wikipedia.org/wiki/Simulated_annealing), entre muchos otrs, FA pretende resolver problemas que estos algoritmos no pueden, dado que
la naturaleza de sus formulación permite separar diferentes subgrupos y buscar diferentes mínimos. Es esta
una de las grandes ventajas del FA pues nunca quedará restringido o _atascado_ en mínimos locales, pues si resulta
que la función tiene diferentes mínimos locales y uno global, el algoritmo está garantizado encontrar un máximo
global al subdividir el grupo total de luciérnagas.

## Descripción Matemática
El FA tiene tres hipótesis o reglas bajo las cuales funciona:
1. Todas las luciérnagas se atraen entre sí, no hay distinción alguna.
2. La atracción es proporcional al brillo, las más brillantes atraen a las menos brillantes.
3. El brillo de cada luciérnaga depende de la _forma_ de la función.

Estas tres reglas describen por completo el algoritmo, esto es las luciérnagas se moverán hacia donde la función
provea el mayor brillo, que corresponde a todos puntos óptimos (máximos o mínimos).

## Atracción
La _atracción_ entre las luciérnagas es algo importante a determinar, pues no todas las funciones tienen formas
semejantes o viven en espacios iguales. Normalmente se considera la forma de la intensidad de luz siguiente

$$\beta(r) = \beta_0 e^{-\gamma r^2}$$

donde $r$ es la distancia euclidiana convencional (aunque esto no siempre es válido, mucho menos en espacios no 
planos), $\gamma$ es un _coeficiente de absorción_ de luz y $\beta_0$ es la intensidad inicial.

Es común emplear esta forma de atracción (es la que se emplea en este documento) sin embargo el autor describe algunas
funciones adicionales dependiendo de las propiedades de la función a estudiar. Se remite al lector al artículo original
para estudiar las diferentes formas propuestas.

## Movimiento y estocasticidad
El movimiento de las luciérnagas es la parte medular de este algoritmo, donde se integra la parte de la atracción según la
intensidad de la luz de cada luciérnaga, así como un elemento aleatorio para facilitar la búsqueda de los óptimos en la función.
Las luciérnagas se mueven de acuerdo a la siguiente expresión

$$\mathbf{x}_i = \mathbf{x}_i + \beta_0 e^{-\gamma r_{ij}^2} (\mathbf{x}_j - \mathbf{x}_i) + \alpha (\text{rand} - \frac{1}{2})$$

donde $\text{rand} \sim U(0, 1)$ y $\alpha \in [0,1]$ es el parámetro que controla la aleatoriedad del FA.

El FA corresponde a la familia de algoritmos de optimización _estocásticos_, esto es que emplean alguna forma de aleatoriedad
para converger más rápido, pero no siempre está garantizada esta convergencia, para esto se emplea el parámetro $\alpha$.
En el caso del FA la convergencia está semi-garantizada por la propiedad de división de grupos de búsqueda en la función,
sin embargo es posible que el algoritmo no converja efectivamente.

## Ventajas y desventajas
Este algoritmo es _eficiente_, con pocas _luciérnagas_ e iteraciones se encuentra el resultado deseado. No requiere de estructuras
computacionales adicionales para ser rápido y simple de implementar. El algoritmo es _multimodal_, y esto ya se ha explicado
en este documento, pero además este algoritmo puede resolver problemas de optimización complicados de tipo NP-duro por su naturaleza
en subdividir grupos y encontrar un resultado más rápido.

Por el contrario, este algoritmo tiene _demasiados_ hiperparámetros por ajustar: 4 en total, considerando que algunos se pueden dejar
siempre iguales y el algoritmo los podrá corregir de forma automática. Este es un problema dado que si los 4 parámetros no están
ajustados de forma adecuada es posible que el algoritmo no converja, en particular el valor de $\alpha$ que controla la
aleatoriedad.

A continuación se presentan funciones de prueba para analizar el algoritmo, así como la implementación.

In [31]:
import numpy as np
import copy
import operator

In [32]:
# Implementación original de Edwin Bedolla

class Firefly:
    def __init__(self, sup, inf, dim):
        self.intensidad = None
        self.posicion = None
        self.lim_sup = sup
        self.lim_inf = inf
        self.dim_fire = dim

    @property
    def pos(self):
        return self.posicion

    @pos.setter
    def pos(self, val):
        self.posicion = val

    @property
    def brillo(self):
        return self.intensidad

    @brillo.setter
    def brillo(self, val):
        self.intensidad = val

    def ini_posicion(self):
        posicion_aleatoria = (self.lim_sup - self.lim_inf) * np.random.random_sample(
            self.dim_fire
        ) + self.lim_inf
        self.posicion = posicion_aleatoria


class FAOpt:
    def __init__(
        self, func, dim, tam_pob, inf, sup, alpha=0.9, beta=0.2, gamma=1.0, args=()
    ):
        self.alpha_opt = alpha
        self.beta_opt = beta
        self.gamma_opt = gamma
        self.lim_sup = sup
        self.lim_inf = inf
        self.func_opt = func
        self.f_args = args
        self.dim_opt = dim
        self.pob_tam = tam_pob
        #  Para crear la población total de posibles soluciones (fireflies) se crea
        # una lista de objetos Firefly con el tamaño especificado por el usuario
        self.poblacion = [Firefly(sup, inf, dim) for i in range(tam_pob)]

    def ini_poblacion(self):
        #  Por cada firefly en la población
        for i in self.poblacion:
            # Asignarle una posición aleatoria
            i.ini_posicion()

    def movimiento(self):
        # Copiar la población anterior
        tmp_poblacion = copy.copy(self.poblacion)

        #  Escala del sistema
        escala = abs(self.lim_sup - self.lim_inf)

        # Iterar entre pares de fireflies, utilizando iteradores de Python
        for i in self.poblacion:
            #  Se necesita el índice de la población anterior actual para cambiar
            #  el valor correspondiente de la población nueva
            for j, k in enumerate(tmp_poblacion):
                # Actualizar la distancia según el algoritmo
                dist = np.power(np.linalg.norm(i.pos - k.pos), 2.0)
                # Si sucede que es una mejor solución
                if i.brillo > k.brillo:
                    #  Implementar el Firefly Algorithm
                    beta_new = (1.0 - self.beta_opt) * np.exp(
                        -self.gamma_opt * dist
                    ) + self.beta_opt
                    estocastico = (
                        self.alpha_opt
                        * (np.random.random_sample(self.dim_opt) - 0.5)
                        * escala
                    )
                    tmp_pos = i.pos * (1.0 - beta_new) + k.pos * beta_new + estocastico

                    # Verificar que las posiciones no se salgan de los límites
                    np.clip(tmp_pos, self.lim_inf, self.lim_sup, out=tmp_pos)

                    #  Actualizar la posición
                    self.poblacion[j].pos = tmp_pos
                    #  Actualizar el brillo de esa posición
                    self.poblacion[j].brillo = -self.func_opt(
                        self.poblacion[j].pos, *self.f_args
                    )

    def optimizar(self, max_gen, optim=False):
        # Inicializar la población, posicion y brillo iniciales
        self.ini_poblacion()
        for i in self.poblacion:
            i.brillo = -self.func_opt(i.pos, *self.f_args)

        # Mover las fireflies tanto como se desee
        for __ in range(max_gen):

            # Ajustar el parámetro alpha
            self.ajuste_alpha(max_gen)

            # Ordernar las fireflies de mayor a menor intensidad
            self.poblacion.sort(key=operator.attrgetter("intensidad"), reverse=True)

            # Mover todas las fireflies
            self.movimiento()

        #  Ordenar la última iteración
        self.poblacion.sort(key=operator.attrgetter("intensidad"), reverse=True)

        # Escoger si se desea obtener el resultado y el valor de la función
        if optim:
            return (self.poblacion[0].pos, self.poblacion[0].brillo)
        else:
            # Sino, regresar la posición más brillante, el resultado
            return self.poblacion[0].pos

    def ajuste_alpha(self, gens):
        #  Se ajusta el valor de alpha según el autor
        delta = 1.0 - (10.0 ** (-4) / 0.9) ** (1.0 / gens)
        self.alpha_opt *= 1.0 - delta



## Pruebas de optimización
A continuación se prueba el algoritmo FA con 3 funciones estándar de optimización para verificar su funcionamiento y validez.

### Función _sphere_
La función _sphere_ está definida como sigue

$$f(\mathbf{x} = \sum_{i=1}^{d} x_i^2$$

donde $d$ es la dimensión del espacio. Esta función tiene como mínimo global $f(\mathbf{x}^{*}) = 0$, $\mathbf{x}^{*} = (0, \cdots, 0)$

En este caso se trabajará con $d = 256$, y se cambiará el origen de la función a 1 para tener el resultado $f(\mathbf{x}^{*}) = 0$, $\mathbf{x}^{*} = (1, \cdots, 1)$.

In [33]:
# Implementar la función Sphere, con el origen movido a 1
def sphere(x):
    return sum((x - 1.0) ** 2)

In [34]:
# Implementar el FA
fa_optim = FAOpt(sphere, 256, 40, -5.12, 5.12, 0.9, 0.2, 1.0)
res, brillo = fa_optim.optimizar(100, optim=True)
print("Firefly algorithm: {0}\n{1}".format(res, -brillo))    

Firefly algorithm: [1.04720685 1.06276613 1.02341733 0.99739175 1.04457852 1.06899109
 1.03413709 1.01786521 1.1098149  1.03794252 0.92872601 0.97747962
 1.02648442 0.92538761 0.97155512 1.07051738 0.98309432 1.00315009
 1.02144401 1.03388843 1.05381604 0.98315669 1.04216554 1.05549468
 1.02314635 0.9561359  0.99149563 1.02590447 0.99421278 1.03551087
 0.98322904 0.96769105 1.06493167 1.05859886 1.04570063 0.99466957
 0.95969733 0.94293161 1.06579188 1.04469271 0.98491056 0.95237589
 1.05659462 1.01100706 1.0627651  0.95795974 0.95667529 0.96724342
 1.07760818 0.99738984 0.93472712 1.09053616 0.99551066 1.00104308
 0.89133734 1.09049238 0.97640135 1.04026925 1.08661644 1.09963588
 1.00553876 1.14351736 1.02128832 1.0024677  0.9762111  0.96170125
 0.92697813 0.9499919  1.00844712 1.06531295 0.93985224 1.07300385
 1.11955244 1.09236731 1.08257133 1.06466633 1.0677103  0.89705377
 1.07170075 0.9967485  1.08782047 1.05102616 1.04501661 0.96346329
 1.02825855 1.01906821 0.98707136 0.9458264

In [35]:
# Implementar la función de Beale
def beale(x):
    # f(3.0, 0.5) = 0.0 es el mínimo
    term1 = (1.5 - x[0] + x[0] * x[1]) ** 2
    term2 = (2.25 - x[0] + x[0] * (x[1] ** 2)) ** 2
    term3 = (2.625 - x[0] + x[0] * (x[1] ** 3)) ** 2

    return term1 + term2 + term3

In [36]:
# Implementar el FA
fa_optim = FAOpt(beale, 2, 25, -4.5, 4.5, 0.9, 0.2, 1.0)
res, brillo = fa_optim.optimizar(35, optim=True)
print("Firefly algorithm: {0}\n{1}".format(res, -brillo))

Firefly algorithm: [3.00317051 0.50172875]
2.2258849972128382e-05


In [37]:
#  Implementar la función de Goldstein-Price
def gp(x):
    # Mínimo en f(0, -1) = 3.0 
    term1 = (x[0] + x[1] + 1.0) ** 2
    term2 = (
        19.0
        - 14.0 * x[0]
        + 3.0 * x[0] ** 2
        - 14.0 * x[1]
        + 6.0 * x[0] * x[1]
        + 3.0 * x[1] ** 2
    )
    term3 = (2.0 * x[0] - 3.0 * x[1]) ** 2
    term4 = (
        18.0
        - 32.0 * x[0]
        + 12.0 * x[0] ** 2
        + 48.0 * x[1]
        - 36.0 * x[0] * x[1]
        + 27.0 * x[1] ** 2
    )

    return (1.0 + term1 * term2) * (30.0 + term3 * term4)

In [38]:
# Implementar el FA
fa_optim = FAOpt(gp, 2, 25, -2.0, 2.0, 0.9, 0.2, 1.0)
res, brillo = fa_optim.optimizar(40, optim=True)
print("Firefly algorithm: {0}\n{1}".format(res, -brillo))

Firefly algorithm: [-0.00179996 -1.00399085]
3.0061717415556624


## Referencias
1. [Artículo original](https://arxiv.org/abs/1003.1466v1). Este es el artículo original del autor donde describe por primera vez el FA.
2. [Artículo posterior](https://arxiv.org/pdf/1308.3898.pdf). Después de desarrollar el algoritmo, el autor decide revisitarlo, ajustando parámetros y mostrando más ejemplos, experimentos y resultados para el algoritmo.